# Using Type Hints (15 mins)

One of Fugue's core philosophies is adapting to data practitioners to let them define their logic in the most native grammar. In effect, this:

* reduces boilerplate code
* reduces framework lock-in
* increase maintainability

## Distributed API calls


In [6]:
import requests as re

res = re.get("https://pokeapi.co/api/v2/pokemon/ditto")
res.json()

{'abilities': [{'ability': {'name': 'limber',
    'url': 'https://pokeapi.co/api/v2/ability/7/'},
   'is_hidden': False,
   'slot': 1},
  {'ability': {'name': 'imposter',
    'url': 'https://pokeapi.co/api/v2/ability/150/'},
   'is_hidden': True,
   'slot': 3}],
 'base_experience': 101,
 'forms': [{'name': 'ditto',
   'url': 'https://pokeapi.co/api/v2/pokemon-form/132/'}],
 'game_indices': [{'game_index': 76,
   'version': {'name': 'red', 'url': 'https://pokeapi.co/api/v2/version/1/'}},
  {'game_index': 76,
   'version': {'name': 'blue', 'url': 'https://pokeapi.co/api/v2/version/2/'}},
  {'game_index': 76,
   'version': {'name': 'yellow',
    'url': 'https://pokeapi.co/api/v2/version/3/'}},
  {'game_index': 132,
   'version': {'name': 'gold', 'url': 'https://pokeapi.co/api/v2/version/4/'}},
  {'game_index': 132,
   'version': {'name': 'silver',
    'url': 'https://pokeapi.co/api/v2/version/5/'}},
  {'game_index': 132,
   'version': {'name': 'crystal',
    'url': 'https://pokeapi.co/api

In [4]:
list_of_apis = ["https://dinosaur-facts-api.shultzlab.com/dinosaurs/random"] * 10

In [ ]:
import pandas as pd
from typing import List

def call_api(items: List[Any]) -> pd.DataFrame:
    names = []
    descriptions = []
    for item in items:
        res = re.get("https://dinosaur-facts-api.shultzlab.com/dinosaurs/random").json()
        names.append(res['Name'])
        descriptions.append(res['Description']

